# 샘플 데이터 생성 6개월   
연습에 사용할 샘플 데이터를 생성합니다.  
- 채널/시간/주중말 인벤 : table-chn, 200 * 24 * 2.  
- 셋탑/지역 인벤 : table-set, 1,000 만. rate, val, req 컬럼 6개월 분.  
- 셋탑/타겟 매핑 정보 : table-map  
- 타겟 인벤 : table-seg, 200.    
  
inventory db 에 생성.  
연산 전에 hive 에서 생성해서 넘겨주는 데이터  

*** 1,000만 6개월 컬럼 샘플 데이터 ***  
100 part  
- 데이터 생성 : 31 초 
- 파키 쓰기 :  68 초 (8 core 32GB, 3 data node) 
- 파키 쓰기 :  120 초 (4 core 16GB, 2 data node) 


In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [2]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
partition_num = 50
tbl_setop_name = f'inven/table-set-6m-{partition_num}-{scale}'
file_format = 'parquet'

PRJ_ROOT = '/user/root'
APP_NAME = f'spark-01-sample-creation-6m-{partition_num}-{scale}'
DB_NAME = 'inven'

In [3]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '2').config('spark.driver.memory', '2g')\
    .config('spark.num.executors', '2')\
    .config('spark.executor.cores', '4').config('spark.executor.memory', '4g').getOrCreate()
    sc = spark.sparkContext
    sc
    return spark

In [4]:
spark = spark_creation()

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [5]:
scale = 1000

In [6]:
# 샘플 데이터 생성 
def create_setops():
    setop_count = scale * 10000
    inv_rate = 1.0 
    # 1개월에 최대 1000 건 청약 가정. 
    inv_val = 1000
    inv_req = 0
    setop_name = ['ST_A', 'ST_B', 'ST_C', 'ST_D', 'ST_E', 'ST_F', 'ST_G', 'ST_H', 'ST_I', 'ST_J']
    setops = []
    for s in setop_name:
        for i in range(0, int(setop_count/len(setop_name))):
            setop_id = f'{s}_{i:07d}'
            setops.append([setop_id, inv_rate,inv_val,inv_req, inv_rate,inv_val,inv_req\
                           , inv_rate,inv_val,inv_req, inv_rate,inv_val,inv_req, inv_rate,inv_val,inv_req, inv_rate,inv_val,inv_req])
            
    print(setops[-2:])
    return setops
# 샘플 데이터 형식 정의. 읽기/쓰기 편의 제공. 
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType
    columns = [
        StructField("setop", StringType())
        , StructField("inv_rate_01", FloatType())
        , StructField("inv_val_01", LongType())
        , StructField("inv_req_01", LongType())
        , StructField("inv_rate_02", FloatType())
        , StructField("inv_val_02", LongType())
        , StructField("inv_req_02", LongType())
        , StructField("inv_rate_03", FloatType())
        , StructField("inv_val_03", LongType())
        , StructField("inv_req_03", LongType())
        , StructField("inv_rate_04", FloatType())
        , StructField("inv_val_04", LongType())
        , StructField("inv_req_04", LongType())
        , StructField("inv_rate_05", FloatType())
        , StructField("inv_val_05", LongType())
        , StructField("inv_req_05", LongType())        
        , StructField("inv_rate_06", FloatType())
        , StructField("inv_val_06", LongType())
        , StructField("inv_req_06", LongType())        
    ]
    sample_schema = StructType(columns)
    return sample_schema

# 샘플 타겟 정보 생성. 20 개 카테고리. 
def create_target():
    segs = []
    seg_count = 20
    for i in range(0, seg_count):
        seg_id = f'CATEGORY_{i:03d}'
        segs.append(seg_id)
            
    print(segs[-2:])
    return segs

In [7]:
%%time 
# 샘플 데이터 생성 및 확인 
sample_data = create_setops()
sample_schema = define_schema()

rdd = spark.sparkContext.parallelize(sample_data, partition_num)
df = spark.createDataFrame(rdd, sample_schema)
#df.show(10)

[['ST_J_0999998', 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0], ['ST_J_0999999', 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0]]
CPU times: user 25.4 s, sys: 2.37 s, total: 27.8 s
Wall time: 31.1 s


In [8]:
%%time 
write_mode = 'overwrite'
# HDFS 에 /user/root/inven/setop 폴더와 파일이 생성 됨.  
# 클러스터 메모리/cpu 여유 있으면, 적당한 크기로 한방에 처리.
df.write.save(path=tbl_setop_name, format=file_format, mode=write_mode)
# # 클러스터 메모리/cpu 적은 경우, 한번에 쓰면 오히려 오래 걸림. 
# df.createOrReplaceTempView('temp')
# print('---- temp view created -----')
# setop_types = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'] 
# for stype in setop_types:
#     df_tmp = spark.sql(f"select * from temp where setop like 'ST_{stype}%'")
#     df.write.save(path=tbl_setop_name, format=file_format, mode=write_mode)
#     write_mode = 'append'
#     print(f' {stype} saved. ')
# print('---------')

CPU times: user 27.8 ms, sys: 5.9 ms, total: 33.7 ms
Wall time: 2min 3s


In [9]:
%%time 
rdd.unpersist()
del rdd

CPU times: user 2.18 ms, sys: 0 ns, total: 2.18 ms
Wall time: 54.6 ms


In [10]:
%%time 
# 기록한 파일 다시 읽어 들이기 
# 저장 결과 확인하기 
lines = spark.read.format(file_format).option('path', tbl_setop_name).load()
data_count = lines.count()
print(f'DATA Count : {data_count:,}')
lines.show(5)

DATA Count : 10,000,000
+------------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|       setop|inv_rate_01|inv_val_01|inv_req_01|inv_rate_02|inv_val_02|inv_req_02|inv_rate_03|inv_val_03|inv_req_03|inv_rate_04|inv_val_04|inv_req_04|inv_rate_05|inv_val_05|inv_req_05|inv_rate_06|inv_val_06|inv_req_06|
+------------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|ST_A_0599040|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      1000|         0|
|ST_A_0599041|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      

In [11]:
# 타겟 : 세톱 매핑 정보 생성  


In [11]:
# 샘플 타겟 정보 생성
# 샘플 타겟 정보 생성
def create_target():
    segs = []
    seg_count = 20
    for i in range(0, seg_count):
        seg_id = f'CATEGORY_{i:03d}'
        segs.append(seg_id)
            
    print(segs[-2:])
    return segs

In [12]:
%%time
# 20 카테고리 세탑 맵을 생성하는데, 약 7분 소요. 2억 건.   
tbl_segmap = f'inven/table-segmap-{partition_num}-{scale}'
# 세톱 데이터를 읽어들인다. 
# 타겟, 세톱 
spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView('setop_view')
# A, B 세탑을 맵에 할당한다.  
## A 세탑을 첫번째 타겟에 할당  
print('--- read data completed. ---')
# 매번 새로운 값으로 덮어쓰기. for 안에 실행되는 부분은 덧붙이기. 
write_mode = 'overwrite'
segs = create_target()
for seg in segs:
    print(f'==== SEG {seg} ====')
    for setop in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'] :
        ### A, setop_id 형태로 생성  
        df = spark.sql(f"select '{seg}' target, setop from setop_view where setop like 'ST_{setop}%'")
        df.write.save(path=tbl_segmap, format=file_format, mode=write_mode)
        write_mode = 'append'
        #print('--- write end  ---')

print('---> SEG/Setop map creation completed. <---')

--- read data completed. ---
['CATEGORY_018', 'CATEGORY_019']
==== SEG CATEGORY_000 ====
==== SEG CATEGORY_001 ====
==== SEG CATEGORY_002 ====
==== SEG CATEGORY_003 ====
==== SEG CATEGORY_004 ====
==== SEG CATEGORY_005 ====
==== SEG CATEGORY_006 ====
==== SEG CATEGORY_007 ====
==== SEG CATEGORY_008 ====
==== SEG CATEGORY_009 ====
==== SEG CATEGORY_010 ====
==== SEG CATEGORY_011 ====
==== SEG CATEGORY_012 ====
==== SEG CATEGORY_013 ====
==== SEG CATEGORY_014 ====
==== SEG CATEGORY_015 ====
==== SEG CATEGORY_016 ====
==== SEG CATEGORY_017 ====
==== SEG CATEGORY_018 ====
==== SEG CATEGORY_019 ====
---> SEG/Setop map creation completed. <---
CPU times: user 368 ms, sys: 88 ms, total: 456 ms
Wall time: 7min 26s


In [13]:
%%time
spark.read.format(file_format).load(tbl_segmap).createOrReplaceTempView('view')
spark.sql("select target, count(1) from view group by target").show()

+------------+--------+
|      target|count(1)|
+------------+--------+
|CATEGORY_013|10000000|
|CATEGORY_016|10000000|
|CATEGORY_014|10000000|
|CATEGORY_006|10000000|
|CATEGORY_009|10000000|
|CATEGORY_010|10000000|
|CATEGORY_004|10000000|
|CATEGORY_005|10000000|
|CATEGORY_018|10000000|
|CATEGORY_000|10000000|
|CATEGORY_017|10000000|
|CATEGORY_012|10000000|
|CATEGORY_002|10000000|
|CATEGORY_001|10000000|
|CATEGORY_019|10000000|
|CATEGORY_008|10000000|
|CATEGORY_003|10000000|
|CATEGORY_015|10000000|
|CATEGORY_007|10000000|
|CATEGORY_011|10000000|
+------------+--------+

CPU times: user 5.03 ms, sys: 3.73 ms, total: 8.76 ms
Wall time: 17.4 s


In [14]:
spark.stop()